# 1. Introduction

This notebook aims to reproduce and explore the methodologies presented in the article "Optimal Bayesian estimation of Gaussian mixtures with growing number of components" by Ilsang Ohn and Lizhen Lin. The focus is on Bayesian estimation of finite Gaussian mixture models (GMMs) where the number of components is unknown and allowed to grow with the sample size.

Gaussian mixture models are fundamental tools in statistical modeling and data analysis, used to represent a wide range of complex data distributions. However, estimating the parameters of GMMs, especially when the number of components is unknown and potentially large, poses significant challenges. Traditional methods may suffer from overfitting or underfitting, leading to inaccurate estimates and poor generalization.

In this notebook, we aim to implement and compare various Bayesian estimation methods for finite Gaussian mixture models, including sample size-dependent priors and advanced computational algorithms.
Investigate the theoretical foundations of these methods, focusing on optimal posterior contraction rates and their implications for model estimation.
Evaluate the performance of these methods through simulation studies under different scenarios, including cases with well-separated components, overlapping components, weak components, and mixtures with a growing number of components.
Apply the developed methods to real-world datasets, such as the Galaxy dataset and the Old Faithful geyser dataset, to demonstrate their practical utility and interpret the results.
Explore methods for estimating the number of components in mixture models, leveraging Bayesian posterior inference and model selection criteria.


# 2. Theoretical Background


In this section, we provide a comprehensive overview of the theoretical foundations underlying the Bayesian estimation of finite Gaussian mixture models (GMMs) with a growing number of components. We introduce key concepts, notations, and mathematical results that are essential for understanding the methodologies implemented in this notebook.

## 2.1 Finite Gaussian Mixture Models


### Definition

A **finite Gaussian mixture model** represents a probability distribution as a convex combination of a finite number of Gaussian (normal) distributions. Formally, a Gaussian mixture model can be expressed as:

$$
p_{\nu ∗ Φ}(x) = \int_{θ ∈ Θ} φ(x - θ) \, \nu(dθ),
$$

where:

- $ x ∈ \mathbb{R} $ is the observed data.
- $ Θ \subseteq \mathbb{R} $ is the parameter space for the component means.
- $ φ(x - θ) $ is the probability density function (pdf) of the normal distribution with mean $ θ $ and variance $ 1 $ (standard normal shifted by $ θ $).
- $ ν $ is the **mixing distribution**, a probability measure on $ Θ $.
- $ ν ∗ Φ $ denotes the convolution of $ ν $ with the standard normal distribution $ Φ $.

When $ ν $ is a discrete measure with finite support, the mixture model becomes finite:

$$
\nu = \sum_{j=1}^k w_j δ_{θ_j},
$$

so that the mixture density simplifies to:

$$
p_{\nu ∗ Φ}(x) = \sum_{j=1}^k w_j φ(x - θ_j),
$$

where:

- $ k ∈ \mathbb{N} $ is the number of components.
- $ w = (w_1, ..., w_k) $ are the **mixing weights**, satisfying $ w_j ≥ 0 $ and $ \sum_{j=1}^k w_j = 1 $.
- $ θ = (θ_1, ..., θ_k) ∈ Θ^k $ are the **component means**.
- $ δ_{θ_j} $ is the Dirac delta measure at $ θ_j $.

### Notations and Definitions

- **Mixing Distribution Space**: Let $ \mathcal{M}(Θ) $ denote the set of all probability measures on $ Θ $. Specifically, when $ Θ = [-L, L] $ for some $ L > 0 $, we write $ \mathcal{M}([-L, L]) $.
- **Finite Mixtures**: Define $ \mathcal{M}_k \subset \mathcal{M}([-L, L]) $ as the subset of mixing distributions that are discrete with at most $ k $ atoms.
- **Data Generation**: Assume we observe $ X_1, X_2, ..., X_n $ independent and identically distributed (i.i.d.) samples from the mixture distribution $ p_{\nu ∗ Φ} $.
- **First-Order Wasserstein Distance**: For probability measures $ μ $ and $ ν $ on $ \mathbb{R} $, the first-order Wasserstein distance is defined as:

  $$
  W_1(μ, ν) = \inf_{γ ∈ \Gamma(μ, ν)} \int_{ℝ × ℝ} |x - y| \, dγ(x, y),
  $$

  where $ \Gamma(μ, ν) $ is the set of all couplings of $ μ $ and $ ν $.

### Identifiability

- **Strong Identifiability**: A mixture model is said to be strongly identifiable if different mixing distributions correspond to different mixture densities.
- **Implications**: For the Gaussian location mixture model with known variance, the model is identifiable under mild conditions. This property is crucial for consistent estimation of the mixing distribution.

### Estimation Goals

- **Estimating the Mixing Distribution**: Our primary goal is to estimate $ ν $ based on the observed data.
- **Estimating the Number of Components**: We also aim to estimate $ k $, the true number of mixture components.
- **Performance Metric**: We evaluate estimation accuracy using the Wasserstein distance $ W_1(ν, \hat{ν}) $ between the true and estimated mixing distributions.



## 2.2 Sample Size-Dependent Priors


### Motivation

In Bayesian inference for mixture models, the choice of prior distribution significantly influences the posterior estimates, especially for the number of components $ k $. A key challenge is to avoid overestimating $ k $ as the sample size $ n $ increases. To address this, we employ **sample size-dependent priors** that penalize models with a large number of components more heavily as $ n $ grows.

### Hierarchical Prior Structure

Our prior distribution $ \Pi $ on the mixing distribution $ ν $ is constructed hierarchically:

1. **Prior on the Number of Components $ k $**: $ \Pi(k) $.
2. **Prior on the Mixing Weights $ w $ Given $ k $**: $ \Pi(w | k) $.
3. **Prior on the Component Means $ θ $ Given $ k $**: $ \Pi(θ | k) $.

### Assumptions on the Prior

We impose the following assumptions (denoted as **(P1)**, **(P2)**, and **(P3)**) on the prior distributions:

#### (P1) Prior on the Number of Components

There exist constants $ A > 0 $, $ c_1 > 0 $, and $ \bar{k}_n = o\left( \frac{\log n}{\log \log n} \right) $ such that for all $ k \leq \bar{k}_n $:

$$
\Pi(k) \geq c_1 e^{-A k \log n}.
$$

This assumption ensures that the prior probability of larger $ k $ decreases exponentially, preventing overestimation as $ n $ increases.

#### (P2) Prior on the Mixing Weights

There exist constants $ κ_0 ∈ (0, 1) $ and $ c_2 > 0 $ such that for any $ k ∈ \mathbb{N} $ and any $ w = (w_1, ..., w_k) $ satisfying $ w_j ≥ κ_0 $ for all $ j $:

$$
\Pi(w | k) \geq c_2.
$$

This ensures that the prior assigns positive probability to weight vectors where each component has a minimum weight, avoiding degenerate cases.

#### (P3) Prior on the Component Means

There exists a constant $ c_3 > 0 $ such that for any $ k ∈ \mathbb{N} $ and any $ θ = (θ_1, ..., θ_k) ∈ [-L, L]^k $:

$$
\Pi(θ | k) \geq c_3^k.
$$

This implies that the prior on $ θ $ is at least as diffuse as a uniform distribution over $ [-L, L]^k $.

### Examples of Priors Satisfying the Assumptions

#### Example 1: Mixture of Finite Mixtures (MFM) Prior

- **Prior on $ k $**: A geometric distribution with success probability $ p_n = 1 - a e^{-A \bar{k}_n \log n} $ satisfies (P1).
- **Prior on $ w $**: A Dirichlet distribution $ \text{Dir}(κ, ..., κ) $ with $ κ ∈ (κ_0, 1) $ satisfies (P2).
- **Prior on $ θ $**: A uniform distribution over $ [-L, L]^k $ satisfies (P3).

#### Example 2: Spike-and-Slab Prior

- **Over-fitted Mixture**: Consider an over-fitted model with $ \bar{k}_n $ components.
- **Prior on Weights**: Assign a spike at zero with probability $ 1 - p_n $ and a slab (e.g., Gamma distribution) with probability $ p_n $.
- **Induced Prior on $ k $**: The number of non-zero weights follows a Binomial distribution, satisfying (P1).

### Benefits of Sample Size-Dependent Priors

- **Control Model Complexity**: By penalizing larger $ k $, we prevent the posterior from favoring overly complex models.
- **Optimal Estimation**: Enables the posterior to concentrate around the true mixing distribution at optimal rates.
- **Avoid Overfitting**: Reduces the risk of overestimating the number of components due to random fluctuations in the data.

## 2.3 Posterior Contraction Rates and Optimality


### Posterior Contraction Rate

The **posterior contraction rate** quantifies how quickly the posterior distribution concentrates around the true parameter value as the sample size $ n $ increases. For estimating the mixing distribution $ ν $, we are interested in the rate $ \epsilon_n $ such that:

$$
E_{P_{ν ∗ Φ}} \left[ \Pi\left( \nu : W_1(ν, ν_0) \geq M \epsilon_n \mid X_1^n \right) \right] \rightarrow 0 \quad \text{as } n \rightarrow \infty,
$$

for some constant $ M > 0 $, where:

- $ ν_0 $ is the true mixing distribution.
- $ X_1^n = (X_1, ..., X_n) $ are the observed data.
- $ E_{P_{ν ∗ Φ}} $ denotes expectation under the true data-generating process.

### Main Theoretical Results

The following are key theoretical results from the article:

#### Theorem 2.1: Posterior Does Not Overestimate $ k $

Under assumptions (P1), (P2), and (P3), and provided $ k \leq \bar{k}_n = o\left( \frac{\log n}{\log \log n} \right) $, we have:

$$
\inf_{ν ∈ \mathcal{M}_k} P_{ν ∗ Φ} \left( \Pi\left( \nu ∈ \mathcal{M}_k \mid X_1^n \right) \rightarrow 1 \right).
$$

**Interpretation**: The posterior probability concentrates on mixing distributions with at most $ k $ components, avoiding overestimation of $ k $.

#### Theorem 2.2: Optimal Posterior Contraction Rate for $ ν $

Under the same assumptions, the posterior contraction rate for estimating $ ν $ is:

$$
\epsilon_n = \left( \frac{\log^2 n}{n} \right)^{\frac{1}{4k - 2}}.
$$

Specifically,

$$
\sup_{ν ∈ \mathcal{M}_k} P_{ν ∗ Φ} \left( \Pi\left( W_1(ν, ν) \geq M \epsilon_n \mid X_1^n \right) = o(1) \right),
$$

for some constant $ M > 0 $.

**Interpretation**: The posterior distribution of $ ν $ concentrates around the true $ ν $ at the optimal rate $ \epsilon_n $.

#### Theorem 2.3: Improved Convergence Under Separation

If the true mixing distribution $ ν $ has well-separated components, the posterior contraction rate improves to:

$$
\epsilon_n = \left( \frac{\log^2 n}{n} \right)^{\frac{1}{4(k - k_0) + 2}},
$$

where $ k_0 \leq k $ is the number of well-separated components.

**Definition of Separation**:

- **Definition 1**: An atomic distribution $ ν = \sum_{j=1}^k w_j δ_{θ_j} $ is said to be **$ k_0 (γ, ω) $-separated** if:
  - There exists a partition $ S_1, ..., S_{k_0} $ of $ \{1, ..., k\} $ such that:
    - $ |θ_j - θ_{j'}| \geq γ $ for any $ j ∈ S_l $, $ j' ∈ S_{l'} $, $ l ≠ l' $.
    - $ \sum_{j ∈ S_l} w_j \geq ω $ for each $ l $.

**Implications**:

- Under separation, we effectively have fewer components ($ k_0 $) to estimate, leading to faster convergence.
- The rate depends on the number of closely clustered components ($ k - k_0 $).

### Minimax Optimality

The rate $ \epsilon_n $ matches the **minimax lower bound** up to logarithmic factors, meaning that no estimator can achieve a substantially faster rate in the worst-case scenario.

- **Minimax Rate**: The best possible rate that any estimator can achieve, considering the most challenging distributions within the model class.
- **Consequence**: Our Bayesian estimator is near-optimal in terms of convergence rate.

### Posterior Consistency for $ k $

Under certain conditions (e.g., perfectly separated components with weights bounded away from zero), the posterior distribution is consistent for the true number of components $ k $:

$$
\inf_{ν ∈ \mathcal{M}_{k, k, γ, ω}} P_{ν ∗ Φ} \left( \Pi\left( ν ∈ \mathcal{M}_k \setminus \mathcal{M}_{k - 1} \mid X_1^n \right) \rightarrow 1 \right),
$$

where $ \mathcal{M}_{k, k, γ, ω} $ denotes the set of mixing distributions that are $ k $-separated with parameters $ γ $ and $ ω $.

### Extension to Higher-Order Mixtures

For models where $ k $ grows with $ n $ (higher-order mixtures), the posterior contraction rate becomes:

$$
\epsilon_n = \frac{\log \log n}{\log n}.
$$

This rate is still minimax optimal for such models.

### Significance of the Results

- **Control of Overfitting**: Sample size-dependent priors prevent the posterior from overestimating the number of components, even as $ n $ grows.
- **Optimal Estimation**: The Bayesian estimator achieves the fastest possible convergence rates under the given model assumptions.
- **Adaptive Estimation**: The method adapts to the true level of complexity (e.g., effective number of components due to separation).

### Practical Implications

- By designing appropriate priors, practitioners can ensure that their Bayesian mixture models perform optimally, without overfitting or underfitting.
- The theoretical guarantees provide confidence in applying these methods to real-world data, where the true number of components may be unknown and potentially large.

---

**Conclusion of Theoretical Background**

In this section, we have established the foundational concepts and mathematical results necessary for implementing and understanding Bayesian estimation methods for Gaussian mixture models with a growing number of components. By utilizing sample size-dependent priors and carefully analyzing posterior contraction rates, we can achieve optimal estimation performance, balancing model complexity and data fitting.


# 3. Implementation of Bayesian Estimation Methods


## 3.1 Sample Size-Dependent Priors


## 3.2 Reversible Jump MCMC (RJMCMC) Sampler


## 3.3 EM Algorithm for Posterior Mode Estimation


## 3.4 Denoised Method of Moments (DMM) Algorithm


## 3.5 Dirichlet Process (DP) Mixture Models



# 4. Simulation Studies


## 4.1 Design of Simulation Scenarios


## 4.2 Implementation of Simulation Experiments


## 4.3 Evaluation Metrics


## 4.4 Visualization



# 5. Estimation of the Number of Components


## 5.1 Bayesian Estimation Using Posterior Distributions


## 5.2 Model Selection Criteria


## 5.3 Cross-Validation


## 5.4 Comparison of Methods



# 6. Real Data Analysis


## 6.1 Galaxy Data


## 6.2 Old Faithful Geyser Data



# 7. Discussion and Conclusions


## 7.1 Summary of Results


## 7.2 Computational Considerations


## 7.3 Methodological Insights


## 7.4 Future Work



# Appendices (Optional)


## A.1 Mathematical Derivations


## A.2 Additional Plots


## A.3 Code Snippets



# Final Notes


## F.1 Reproducibility


## F.2 References


## F.3 Collaboration and Sharing